# Particle Swarm Optimization Basics
https://deap.readthedocs.io/en/master/examples/pso_basic.html
https://github.com/DEAP/deap/blob/f141d3fe690e85a2748dae5b73f14ad5e9f784ad/examples/pso/basic.py

In [2]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

import operator
import random

import numpy

from deap import base
from deap import benchmarks
from deap import creator
from deap import tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Particle", list, fitness=creator.FitnessMax, speed=list, 
    smin=None, smax=None, best=None)


def generate(size, pmin, pmax, smin, smax):
    part = creator.Particle(random.uniform(pmin, pmax) for _ in range(size)) 
    part.speed = [random.uniform(smin, smax) for _ in range(size)]
    part.smin = smin
    part.smax = smax
    return part

def updateParticle(part, best, phi1, phi2):
    u1 = (random.uniform(0, phi1) for _ in range(len(part)))
    u2 = (random.uniform(0, phi2) for _ in range(len(part)))
    v_u1 = map(operator.mul, u1, map(operator.sub, part.best, part))
    v_u2 = map(operator.mul, u2, map(operator.sub, best, part))
    part.speed = list(map(operator.add, part.speed, map(operator.add, v_u1, v_u2)))
    for i, speed in enumerate(part.speed):
        if speed < part.smin:
            part.speed[i] = part.smin
        elif speed > part.smax:
            part.speed[i] = part.smax
    part[:] = list(map(operator.add, part, part.speed))

toolbox = base.Toolbox()
toolbox.register("particle", generate, size=2, pmin=-6, pmax=6, smin=-3, smax=3)
toolbox.register("population", tools.initRepeat, list, toolbox.particle)
toolbox.register("update", updateParticle, phi1=2.0, phi2=2.0)
toolbox.register("evaluate", benchmarks.h1)

def main():
    pop = toolbox.population(n=5)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = ["gen", "evals"] + stats.fields

    GEN = 1000
    best = None

    for g in range(GEN):
        for part in pop:
            part.fitness.values = toolbox.evaluate(part)
            if not part.best or part.best.fitness < part.fitness:
                part.best = creator.Particle(part)
                part.best.fitness.values = part.fitness.values
            if not best or best.fitness < part.fitness:
                best = creator.Particle(part)
                best.fitness.values = part.fitness.values
        for part in pop:
            toolbox.update(part, best)

        # Gather all the fitnesses in one list and print the stats
        logbook.record(gen=g, evals=len(pop), **stats.compile(pop))
        print(logbook.stream)
    
    return pop, logbook, best

if __name__ == "__main__":
    main()


C:\Program Files\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


gen	evals	avg      	std      	min       	max     
0  	5    	0.0833498	0.0803221	0.00264376	0.220763
1  	5    	0.123993 	0.120937 	0.0298424 	0.356577
2  	5    	0.152211 	0.0479654	0.106863  	0.243769
3  	5    	0.278765 	0.155797 	0.00154242	0.456871
4  	5    	0.285554 	0.164294 	0.0546981 	0.449916
5  	5    	0.290367 	0.13261  	0.125197  	0.528531
6  	5    	0.434296 	0.349741 	0.188222  	1.11292 
7  	5    	0.337474 	0.124742 	0.116049  	0.493379
8  	5    	0.343103 	0.103518 	0.182855  	0.457023
9  	5    	0.325653 	0.235068 	0.0579681 	0.723708
10 	5    	0.248783 	0.11537  	0.10254   	0.406947
11 	5    	0.553493 	0.453305 	0.0186451 	1.34721 
12 	5    	0.337284 	0.146172 	0.0767251 	0.46724 
13 	5    	0.378887 	0.185427 	0.19453   	0.635615
14 	5    	0.347743 	0.111143 	0.200246  	0.475323
15 	5    	0.525805 	0.488684 	0.223801  	1.49908 
16 	5    	0.765797 	0.53106  	0.184582  	1.52205 
17 	5    	0.235735 	0.143641 	0.00550991	0.459163
18 	5    	0.34997  	0.242096 	0.0195682 	0.706902
